# Deploying ML module on IoT Edge Device

In [300]:
import requests
import json
import time

import docker
import matplotlib.pyplot as plt
import numpy as np
import requests
from azure.mgmt.containerregistry import ContainerRegistryManagementClient
from azureml.core.workspace import Workspace
from dotenv import set_key, get_key, find_dotenv
from testing_utilities import to_img, read_image_from, plot_predictions, get_auth


In [301]:
# get the .env file where all the variables are stored
env_path = find_dotenv(raise_error_if_not_found=True)

In [ ]:
resource_group = get_key(env_path, 'resource_group')
iot_hub_name = get_key(env_path, 'iot_hub_name')
device_id = get_key(env_path, 'device_id')


acr_name = get_key(env_path, 'acr_name')
acr_login_server = get_key(env_path, 'acr_login_server')
acr_password = get_key(env_path, 'acr_password')
mlimg_location = get_key(env_path, 'ml_img_location')

In [ ]:
# deploy ML module
ml_module_name = 'image-classification-service'

In [ ]:
# get the image from workspace
ws = Workspace.from_config(auth=get_auth())
image_name = get_key(env_path, 'image_name')
image = ws.images[image_name]

In [ ]:
# Getting your container details
container_reg = ws.get_details()["containerRegistry"]
reg_name=container_reg.split("/")[-1]
acr_login_server=''.join([reg_name, '.azurecr.io'])
subscription_id = ws.subscription_id

client = ContainerRegistryManagementClient(ws._auth,subscription_id)
result= client.registries.list_credentials(resource_group, reg_name, custom_headers=None, raw=False)
acr_username = result.username
acr_password = result.passwords[0].value
print('ContainerURL:{}'.format(image.image_location))
print('Servername: {}'.format(acr_login_server))
print('Username: {}'.format(acr_username))
print('Password: {}'.format(acr_password))

In [346]:
file = open('./deployment_MLmodule_template.json')
contents = file.read()
    
contents = contents.replace('__REGISTRY_SERVER_NAME', acr_login_server)
contents = contents.replace('__REGISTRY_PASSWORD', acr_password)
contents = contents.replace('__REGISTRY_USER_NAME', acr_username)
contents = contents.replace('__ML_MODULE_NAME', ml_module_name)
contents = contents.replace('__REGISTRY_ML_IMAGE_LOCATION', image.image_location)

with open('./deployment.json', 'wt', encoding='utf-8') as output_file:
    output_file.write(contents)

In [ ]:
print("start deployment process...")

In [ ]:
# Push the deployment JSON to the IOT Hub
!az iot edge set-modules --device-id $device_id \
                         --hub-name $iot_hub_name \
                         --content deployment.json \
                         -g $resource_group

In [ ]:
docker_client = docker.APIClient(base_url='unix://var/run/docker.sock')

In [348]:
def get_id(container_name):
    contents = docker_client.containers()
    for cont in contents:
        if container_name in cont['Names'][0]:
            return cont["Id"]
    return None

In [349]:
d_id = get_id(ml_module_name)
while d_id is None:
    d_id = get_id(ml_module_name)
    time.sleep(1)

In [350]:
logs = docker_client.attach(d_id, stream=True, logs=True)

In [ ]:
# keep running this cell until the log contains "Using TensorFlow backend", which indicates the container is up running.
for l in logs:
    msg = l.decode('utf-8')
    print(msg)
    if "Using TensorFlow backend" in msg:
        break        

When you run `docker ps` command in the edge device, you should see there are three containers running: `edgeAgent`, `edgeHub`, and the container with name `ml_module_name`.

In [ ]:
!docker ps

When the container `ml_module_name` is running, you can also test this module. We show how to do it at the notebook [05_TestMLModule.ipynb](./05_TestMLModule.ipynb).